In [ ]:
import openai
import os
import pyaudio
import wave
from dotenv import load_dotenv
import requests
import json

# Set your OpenAI API key
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY;

In [ ]:
# Record audio
def record_audio(filename, duration=5):
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    CHUNK = 1024

    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("Recording...")

    frames = []

    for _ in range(0, int(RATE / CHUNK * duration)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

In [ ]:
# Transcribe audio
def transcribe_audio(filename):
    with open(filename, "rb") as audio_file:
        transcript = openai.Audio.transcribe("whisper-1", audio_file)
        return transcript["text"]

In [ ]:
def generate_response(question):
    # Defina os cabeçalhos da solicitação
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }
    
    # Defina os dados da solicitação
    data = {
        "model": "gpt-4",
        "messages": [{"role": "user", "content": f"{question}"}],
        "temperature": 0.7
    }
    
    # Faça a solicitação HTTP POST
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=data)
    
    # Obtenha a resposta da API
    result = response.json()
    
    # Acesse a resposta do modelo
    return result["choices"][0]["message"]["content"]

In [ ]:
# Main function
def main():
    audio_filename = "recorded_audio.wav"
    record_audio(audio_filename)
    transcription = transcribe_audio(audio_filename)
    print("Transcription:", transcription)
    response = generate_response(transcription)
    print(response)
if __name__ == "__main__":
    main()